In [1]:
!pip install flask langchain langgraph openai langchain-community langchain-openai geopy timezonefinder dateparser pytz google-api-python-client google-auth-httplib2 google-auth-oauthlib --quiet

In [1]:
# Import all necessary libraries
from flask import Flask, request, jsonify
from threading import Thread
import json
from datetime import datetime, timezone, timedelta
import pytz
from collections import defaultdict
import re

# LangChain and LangGraph imports
from langchain_openai import ChatOpenAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langgraph.graph import StateGraph, END

# Google Calendar API imports
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

# Location and timezone utilities
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder

# Initialize Flask app
app = Flask(__name__)
received_data = []

# Initialize LLM
llm = ChatOpenAI(
    base_url="http://localhost:3000/v1",
    api_key="NULL",
    model="/home/user/Models/deepseek-ai/deepseek-llm-7b-chat",
    temperature=0.0
)

# Helper Functions
def parse_datetime_string(dt_str):
    """Parse datetime string in DD-MM-YYYY format"""
    try:
        # Handle DD-MM-YYYY format
        if 'T' in dt_str:
            date_part, time_part = dt_str.split('T')
            day, month, year = date_part.split('-')
            return datetime.strptime(f"{year}-{month}-{day}T{time_part}", "%Y-%m-%dT%H:%M:%S")
        else:
            day, month, year = dt_str.split('-')
            return datetime.strptime(f"{year}-{month}-{day}", "%Y-%m-%d")
    except:
        # Fallback to ISO format
        return datetime.fromisoformat(dt_str.replace('T', ' '))

def get_timezone(params: dict) -> dict:
    """Get timezone from location"""
    location = params.get("Location")
    
    geolocator = Nominatim(user_agent="calendar_ai_agent")
    location_obj = geolocator.geocode(location)
    if not location_obj:
        return {**params, "timezone": "UTC"}

    tf = TimezoneFinder()
    timezone = tf.timezone_at(lng=location_obj.longitude, lat=location_obj.latitude)
    return {**params, "timezone": timezone or "UTC"}

def extract_specific_time(content):
    """Extract specific time from email content"""
    # Common time patterns
    time_patterns = [
        r'(?:at\s+)?(\d{1,2}):(\d{2})\s*(AM|PM|am|pm)',
        r'(?:at\s+)?(\d{1,2})\s*(AM|PM|am|pm)',
        r'(?:at\s+)?(\d{1,2}):(\d{2})'
    ]
    
    for pattern in time_patterns:
        match = re.search(pattern, content, re.IGNORECASE)
        if match:
            if len(match.groups()) == 3:  # HH:MM AM/PM
                hour, minute, meridiem = match.groups()
                hour = int(hour)
                minute = int(minute)
                if meridiem.upper() == 'PM' and hour != 12:
                    hour += 12
                elif meridiem.upper() == 'AM' and hour == 12:
                    hour = 0
                return f"{hour:02d}:{minute:02d}"
            elif len(match.groups()) == 2:  # HH AM/PM
                hour, meridiem = match.groups()
                hour = int(hour)
                if meridiem.upper() == 'PM' and hour != 12:
                    hour += 12
                elif meridiem.upper() == 'AM' and hour == 12:
                    hour = 0
                return f"{hour:02d}:00"
            else:  # HH:MM (24-hour)
                hour, minute = match.groups()
                return f"{int(hour):02d}:{int(minute):02d}"
    
    return None

def calculate_target_date(reference_dt, content):
    """Calculate target date from email content"""
    content_lower = content.lower()
    
    # Map weekday names to numbers
    weekdays = {
        'monday': 0, 'tuesday': 1, 'wednesday': 2, 'thursday': 3,
        'friday': 4, 'saturday': 5, 'sunday': 6
    }
    
    # Check for relative date keywords
    if 'today' in content_lower:
        return reference_dt.date()
    elif 'tomorrow' in content_lower:
        return (reference_dt + timedelta(days=1)).date()
    
    # Check for specific dates (DD-MM-YYYY or DD/MM/YYYY)
    date_pattern = r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})'
    date_match = re.search(date_pattern, content)
    if date_match:
        day, month, year = date_match.groups()
        try:
            return datetime(int(year), int(month), int(day)).date()
        except ValueError:
            pass
    
    # Check for weekday mentions
    target_weekday = None
    is_next_week = False
    is_this_week = False
    
    # Check for "next" or "this" keywords
    if re.search(r'\bnext\s+\w+day\b', content_lower):
        is_next_week = True
    elif re.search(r'\bthis\s+\w+day\b', content_lower):
        is_this_week = True
    
    # Find the weekday mentioned
    for day_name, day_num in weekdays.items():
        if day_name in content_lower:
            target_weekday = day_num
            break
    
    if target_weekday is not None:
        current_weekday = reference_dt.weekday()
        days_ahead = target_weekday - current_weekday
        
        if is_next_week:
            # Force next week
            if days_ahead <= 0:
                days_ahead += 7
            else:
                days_ahead += 7
        elif is_this_week:
            # Force this week if possible
            if days_ahead < 0:
                days_ahead += 7
        else:
            # Default: next occurrence of the weekday
            if days_ahead <= 0:
                days_ahead += 7
        
        target_date = reference_dt + timedelta(days=days_ahead)
        return target_date.date()
    
    # Default fallback: tomorrow
    return (reference_dt + timedelta(days=1)).date()

def get_meet_details(params: dict) -> dict:
    """Extract meeting details with improved parsing"""
    timezone_str = params.get("timezone")
    dt_str = params.get("Datetime")
    subject = params.get("Subject", "")
    content = params.get("EmailContent", "")

    # Parse the reference datetime
    reference_dt = parse_datetime_string(dt_str)
    
    # Extract specific time if available
    specific_time = extract_specific_time(content)
    
    # Calculate target date
    target_date = calculate_target_date(reference_dt, content)
    
    # Extract duration
    duration_mins = "60"  # Default to 60 minutes
    duration_patterns = [
        r'(\d+)\s*(?:minute|min|mins)',
        r'(\d+)\s*(?:hour|hr|hrs)',
        r'for\s+(\d+)\s*(?:minute|min|mins)',
        r'for\s+(\d+)\s*(?:hour|hr|hrs)'
    ]
    
    for pattern in duration_patterns:
        match = re.search(pattern, content, re.IGNORECASE)
        if match:
            duration_value = int(match.group(1))
            if 'hour' in pattern or 'hr' in pattern:
                duration_value *= 60
            duration_mins = str(duration_value)
            break
    
    # Create timezone-aware datetime objects
    tz = pytz.timezone(timezone_str)
    
    if specific_time:
        # Use specific time
        hour, minute = map(int, specific_time.split(':'))
        start_dt = tz.localize(datetime.combine(target_date, datetime.min.time().replace(hour=hour, minute=minute)))
    else:
        # Use time of day logic
        time_of_day = "morning"  # Default
        if any(word in content.lower() for word in ['afternoon', 'lunch']):
            time_of_day = "afternoon"
        elif any(word in content.lower() for word in ['evening', 'night']):
            time_of_day = "evening"
        
        time_ranges = {
            "morning": 9,
            "afternoon": 14,
            "evening": 17
        }
        
        default_hour = time_ranges.get(time_of_day, 9)
        start_dt = tz.localize(datetime.combine(target_date, datetime.min.time().replace(hour=default_hour)))
    
    # Calculate end time
    end_dt = start_dt + timedelta(minutes=int(duration_mins))
    
    print(f"📅 Meeting details extracted:")
    print(f"   Date: {target_date}")
    print(f"   Time: {start_dt.strftime('%H:%M')}")
    print(f"   Duration: {duration_mins} minutes")
    print(f"   Start: {start_dt.isoformat()}")
    print(f"   End: {end_dt.isoformat()}")
    
    result = {
        **params,
        "Start": start_dt.isoformat(),
        "End": end_dt.isoformat(),
        "Duration_mins": duration_mins
    }
    result.pop("timezone", None)
    return result

def retrieve_calendar_events(user, start, end):
    """Retrieve calendar events for a user"""
    events_list = []
    try:
        token_path = f"./Keys/{user.split('@')[0]}.token"
        user_creds = Credentials.from_authorized_user_file(token_path)
        calendar_service = build("calendar", "v3", credentials=user_creds)
        events_result = calendar_service.events().list(
            calendarId='primary', 
            timeMin=start,
            timeMax=end,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        events = events_result.get('items', [])
        
        for event in events:
            attendee_list = []
            try:
                for attendee in event.get("attendees", []):
                    attendee_list.append(attendee['email'])
            except:
                attendee_list.append("SELF")
            
            if not attendee_list:
                attendee_list.append("SELF")
                
            start_time = event["start"].get("dateTime", event["start"].get("date"))
            end_time = event["end"].get("dateTime", event["end"].get("date"))
            
            events_list.append({
                "StartTime": start_time,
                "EndTime": end_time,
                "NumAttendees": len(set(attendee_list)),
                "Attendees": attendee_list,
                "Summary": event.get("summary", "No Title")
            })
    except Exception as e:
        print(f"Error retrieving calendar for {user}: {e}")
    
    return events_list

def get_priority(email_content):
    """Get meeting priority"""
    content_lower = email_content.lower()
    
    # High priority indicators
    high_priority_words = [
        'urgent', 'asap', 'immediately', 'critical', 'emergency', 
        'at the earliest', 'right away', 'quick', 'promptly'
    ]
    
    if any(word in content_lower for word in high_priority_words):
        return "high"
    
    # Medium priority: specific time/date mentioned
    if any(word in content_lower for word in ['today', 'tomorrow', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday']):
        return "medium"
    
    return "low"

def parse_event_times(events, tzinfo):
    """Parse event times to datetime objects"""
    parsed = []
    for e in events:
        try:
            start = datetime.fromisoformat(e["StartTime"]).astimezone(tzinfo)
            end = datetime.fromisoformat(e["EndTime"]).astimezone(tzinfo)
            parsed.append((start, end))
        except Exception as ex:
            print(f"Error parsing event time: {ex}")
            continue
    return parsed

def merge_intervals(intervals):
    """Merge overlapping time intervals"""
    if not intervals:
        return []
    intervals.sort()
    merged = []
    for interval in intervals:
        if not merged or merged[-1][1] < interval[0]:
            merged.append(interval)
        else:
            merged[-1] = (merged[-1][0], max(merged[-1][1], interval[1]))
    return merged

def fetch_all_attendee_events(emails, day_start, day_end):
    """Fetch events for all attendees"""
    attendee_events = defaultdict(list)
    for email in emails:
        try:
            events = retrieve_calendar_events(email, day_start.isoformat(), day_end.isoformat())
            attendee_events[email] = events
        except Exception as e:
            print(f"❌ Error fetching calendar for {email}: {e}")
    return attendee_events

def check_time_slot_availability(requested_start, requested_end, emails, priority, tz):
    """Check if the requested time slot is available for all attendees"""
    # Get events for the requested day
    day_start = requested_start.replace(hour=0, minute=0, second=0)
    day_end = requested_start.replace(hour=23, minute=59, second=59)
    
    attendee_events = fetch_all_attendee_events(emails, day_start, day_end)
    
    # Check for conflicts
    all_busy = []
    for email, events in attendee_events.items():
        all_busy.extend(parse_event_times(events, tz))
    
    # Check if requested slot conflicts with existing events
    for busy_start, busy_end in all_busy:
        if requested_start < busy_end and requested_end > busy_start:
            print(f"⚠️ Conflict found: {busy_start} - {busy_end}")
            return False
    
    print(f"✅ Requested time slot is available")
    return True

def find_alternative_slot(requested_start, duration, emails, priority, tz):
    """Find alternative time slot if requested time is not available"""
    # Try slots around the requested time
    slot_duration = timedelta(minutes=duration)
    
    # Define search windows based on priority
    if priority == "high":
        search_slots = [
            (requested_start - timedelta(hours=1), requested_start - timedelta(hours=1) + slot_duration),
            (requested_start + timedelta(hours=1), requested_start + timedelta(hours=1) + slot_duration),
            (requested_start - timedelta(hours=2), requested_start - timedelta(hours=2) + slot_duration),
            (requested_start + timedelta(hours=2), requested_start + timedelta(hours=2) + slot_duration),
        ]
    else:
        search_slots = [
            (requested_start + timedelta(hours=1), requested_start + timedelta(hours=1) + slot_duration),
            (requested_start - timedelta(hours=1), requested_start - timedelta(hours=1) + slot_duration),
        ]
    
    # Get events for the day
    day_start = requested_start.replace(hour=0, minute=0, second=0)
    day_end = requested_start.replace(hour=23, minute=59, second=59)
    attendee_events = fetch_all_attendee_events(emails, day_start, day_end)
    
    all_busy = []
    for email, events in attendee_events.items():
        all_busy.extend(parse_event_times(events, tz))
    
    # Check each alternative slot
    for slot_start, slot_end in search_slots:
        # Ensure slot is within business hours (8 AM - 8 PM)
        if slot_start.hour < 8 or slot_end.hour > 20:
            continue
            
        # Check for conflicts
        conflict = False
        for busy_start, busy_end in all_busy:
            if slot_start < busy_end and slot_end > busy_start:
                conflict = True
                break
        
        if not conflict:
            print(f"🔄 Alternative slot found: {slot_start} - {slot_end}")
            return slot_start, slot_end
    
    print(f"❌ No alternative slots found")
    return requested_start, requested_start + slot_duration

def build_processed_input_agent():
    """Build the LangGraph agent"""
    graph = StateGraph(dict)

    graph.add_node("get_timezone", get_timezone)
    graph.add_node("get_meet_details", get_meet_details)

    graph.set_entry_point("get_timezone")
    graph.add_edge("get_timezone", "get_meet_details")
    graph.add_edge("get_meet_details", END)

    return graph.compile()

def your_meeting_assistant(data):
    """Main meeting assistant function"""
    try:
        # Step 1: Process input with LangGraph
        agent = build_processed_input_agent()
        output = agent.invoke(data)
        
        processed_data = output
        print(f"✅ Processed data: {processed_data}")

        # Step 2: Get priority
        priority = get_priority(processed_data["EmailContent"])
        print(f"📌 Priority extracted: {priority}")

        # Step 3: Check availability and find slots
        requested_start = datetime.fromisoformat(processed_data["Start"])
        requested_end = datetime.fromisoformat(processed_data["End"])
        duration = int(processed_data["Duration_mins"])
        
        tz = requested_start.tzinfo
        emails = [processed_data["From"]] + [a["email"] for a in processed_data["Attendees"]]

        print(f"🔍 Checking availability:")
        print(f"  - Emails: {emails}")
        print(f"  - Requested start: {requested_start}")
        print(f"  - Requested end: {requested_end}")
        print(f"  - Duration: {duration} mins")

        # Check if requested time is available
        if check_time_slot_availability(requested_start, requested_end, emails, priority, tz):
            start, end = requested_start, requested_end
            print(f"✅ Using requested time slot")
        else:
            start, end = find_alternative_slot(requested_start, duration, emails, priority, tz)
            print(f"🔄 Using alternative time slot")

        print(f"🎯 Final slot: {start} to {end}")

        # Step 4: Fetch all attendee events for the final output
        attendee_events = fetch_all_attendee_events(emails, start, end)

        # Step 5: Build final output
        attendees_final = []
        for email in emails:
            user_events_raw = attendee_events.get(email, [])
            user_events = []

            # Process existing events
            for e in user_events_raw:
                user_event = {
                    "StartTime": e["StartTime"],
                    "EndTime": e["EndTime"],
                    "NumAttendees": e.get("NumAttendees", 1),
                    "Attendees": e.get("Attendees", ["SELF"]),
                    "Summary": e.get("Summary", "Other Meeting")
                }
                user_events.append(user_event)

            # Add the new meeting
            common_event_user = {
                "StartTime": start.isoformat(),
                "EndTime": end.isoformat(),
                "NumAttendees": len(emails),
                "Attendees": emails,
                "Summary": processed_data.get("Subject", "Meeting")
            }

            user_events.append(common_event_user)

            attendees_final.append({
                "email": email,
                "events": user_events
            })

        # Step 6: Format final output
        final_output = {
            "Request_id": data["Request_id"],
            "Datetime": data["Datetime"],
            "Location": data["Location"],
            "From": data["From"],
            "Attendees": attendees_final,
            "Subject": data.get("Subject", "Meeting"),
            "EmailContent": data["EmailContent"],
            "EventStart": start.isoformat(),
            "EventEnd": end.isoformat(),
            "Duration_mins": str(duration),
            "MetaData": {}
        }

        print(f"✅ Final output generated successfully")
        return final_output

    except Exception as e:
        print(f"❌ Error in meeting assistant: {e}")
        import traceback
        traceback.print_exc()
        
        # Return minimal structure on error
        return {
            **data,
            "EventStart": "",
            "EventEnd": "",
            "Duration_mins": "60",
            "MetaData": {"error": str(e)}
        }

In [2]:
@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()
    print(f"\n Received: {json.dumps(data, indent=2)}")
    new_data = your_meeting_assistant(data)  # Your AI Meeting Assistant Function Call
    received_data.append(data)
    print(f"\n\n\n Sending:\n {json.dumps(new_data, indent=2)}")
    return jsonify(new_data)

def run_flask():
    app.run(host='0.0.0.0', port=5000)

In [3]:
# Start Flask in a background thread
Thread(target=run_flask, daemon=True).start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://134.199.197.82:5000
Press CTRL+C to quit



 Received: {
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033b5",
  "Datetime": "02-07-2025T12:34:55",
  "Location": "IIT Mumbai",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com"
    },
    {
      "email": "userthree.amd@gmail.com"
    }
  ],
  "EmailContent": "Hi Team. We\u2019ve just received quick feedback from the client indicating that the instructions we provided aren\u2019t working on their end. Let\u2019s prioritize resolving this promptly. Let\u2019s meet Monday at 9:00 AM to discuss and resolve this issue."
}
📅 Meeting details extracted:
   Date: 2025-07-07
   Time: 09:00
   Duration: 60 minutes
   Start: 2025-07-07T09:00:00+05:30
   End: 2025-07-07T10:00:00+05:30
✅ Processed data: {'Request_id': '6118b54f-907b-4451-8d48-dd13d76033b5', 'Datetime': '02-07-2025T12:34:55', 'Location': 'IIT Mumbai', 'From': 'userone.amd@gmail.com', 'Attendees': [{'email': 'usertwo.amd@gmail.com'}, {'email': 'userthree.amd@gmail.com'}], 'EmailC

127.0.0.1 - - [13/Jul/2025 08:56:49] "POST /receive HTTP/1.1" 200 -


✅ Final output generated successfully



 Sending:
 {
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033b5",
  "Datetime": "02-07-2025T12:34:55",
  "Location": "IIT Mumbai",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "userone.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-07T09:00:00+05:30",
          "EndTime": "2025-07-07T10:00:00+05:30",
          "NumAttendees": 3,
          "Attendees": [
            "userone.amd@gmail.com",
            "usertwo.amd@gmail.com",
            "userthree.amd@gmail.com"
          ],
          "Summary": "Meeting"
        }
      ]
    },
    {
      "email": "usertwo.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-07T09:00:00+05:30",
          "EndTime": "2025-07-07T10:00:00+05:30",
          "NumAttendees": 3,
          "Attendees": [
            "userone.amd@gmail.com",
            "usertwo.amd@gmail.com",
            "userthree.amd@gmail.com"
          ],
       

...

✅ Final output generated successfully



 Sending:
 {
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033b5",
  "Datetime": "02-07-2025T12:34:55",
  "Location": "IIT Mumbai",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "userone.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-07T09:00:00+05:30",
          "EndTime": "2025-07-07T10:00:00+05:30",
          "NumAttendees": 3,
          "Attendees": [
            "userone.amd@gmail.com",
            "usertwo.amd@gmail.com",
            "userthree.amd@gmail.com"
          ],
          "Summary": "Meeting"
        }
      ]
    },
    {
      "email": "usertwo.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-07T09:00:00+05:30",
          "EndTime": "2025-07-07T10:00:00+05:30",
          "NumAttendees": 3,
          "Attendees": [
            "userone.amd@gmail.com",
            "usertwo.amd@gmail.com",
            "userthree.amd@gmail.com"
          ],
       

127.0.0.1 - - [13/Jul/2025 09:01:24] "POST /receive HTTP/1.1" 200 -


✅ Final output generated successfully



 Sending:
 {
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033c5",
  "Datetime": "02-07-2025T12:34:55",
  "Location": "IIT Mumbai",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "userone.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-08T11:00:00+05:30",
          "EndTime": "2025-07-08T12:00:00+05:30",
          "NumAttendees": 3,
          "Attendees": [
            "userone.amd@gmail.com",
            "usertwo.amd@gmail.com",
            "userthree.amd@gmail.com"
          ],
          "Summary": "Meeting"
        }
      ]
    },
    {
      "email": "usertwo.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-08T11:00:00+05:30",
          "EndTime": "2025-07-08T12:00:00+05:30",
          "NumAttendees": 3,
          "Attendees": [
            "userone.amd@gmail.com",
            "usertwo.amd@gmail.com",
            "userthree.amd@gmail.com"
          ],
       

127.0.0.1 - - [13/Jul/2025 09:03:24] "POST /receive HTTP/1.1" 200 -


✅ Final output generated successfully



 Sending:
 {
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033d5",
  "Datetime": "02-07-2025T12:34:55",
  "Location": "IIT Mumbai",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "userone.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-09T10:00:00+05:30",
          "EndTime": "2025-07-09T11:00:00+05:30",
          "NumAttendees": 3,
          "Attendees": [
            "userone.amd@gmail.com",
            "usertwo.amd@gmail.com",
            "userthree.amd@gmail.com"
          ],
          "Summary": "Meeting"
        }
      ]
    },
    {
      "email": "usertwo.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-09T10:00:00+05:30",
          "EndTime": "2025-07-09T11:00:00+05:30",
          "NumAttendees": 3,
          "Attendees": [
            "userone.amd@gmail.com",
            "usertwo.amd@gmail.com",
            "userthree.amd@gmail.com"
          ],
       

In [10]:
import requests

url = "http://127.0.0.1:5000/receive"

data = {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033d5",
    "Datetime": "02-07-2025T12:34:55",
    "Location": "IIT Mumbai",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "EmailContent": "Hi Team. We’ve received the final feedback from the client. Let’s review it together and plan next steps. Let's meet on Wednesday at 10:00 A.M."
}

response = requests.post(url, json=data)

print("Status Code:", response.status_code)
print("Response JSON:", response.json())

Status Code: 200
Response JSON: {'Attendees': [{'email': 'userone.amd@gmail.com', 'events': [{'Attendees': ['userone.amd@gmail.com', 'usertwo.amd@gmail.com', 'userthree.amd@gmail.com'], 'EndTime': '2025-07-09T11:00:00+05:30', 'NumAttendees': 3, 'StartTime': '2025-07-09T10:00:00+05:30', 'Summary': 'Meeting'}]}, {'email': 'usertwo.amd@gmail.com', 'events': [{'Attendees': ['userone.amd@gmail.com', 'usertwo.amd@gmail.com', 'userthree.amd@gmail.com'], 'EndTime': '2025-07-09T11:00:00+05:30', 'NumAttendees': 3, 'StartTime': '2025-07-09T10:00:00+05:30', 'Summary': 'Meeting'}]}, {'email': 'userthree.amd@gmail.com', 'events': [{'Attendees': ['userone.amd@gmail.com', 'usertwo.amd@gmail.com', 'userthree.amd@gmail.com'], 'EndTime': '2025-07-09T11:00:00+05:30', 'NumAttendees': 3, 'StartTime': '2025-07-09T10:00:00+05:30', 'Summary': 'Meeting'}]}], 'Datetime': '02-07-2025T12:34:55', 'Duration_mins': '60', 'EmailContent': "Hi Team. We’ve received the final feedback from the client. Let’s review it toge